Мадина Светлана Владимировна, РИ-400034

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%timeit

df_pl = pl.read_csv("train.csv")

918 µs ± 5.97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
import polars as pl

# Загрузка датасета
df = pl.read_csv("train.csv")

# Вывод типов данных столбцов
print("Типы данных столбцов:")
print(df.schema)

# Вывод общей информации о датасете, включая количество пропусков
print("\nОбщая информация о датасете:")
print(df.describe())

# Для подсчета количества пропусков в каждом столбце можно использовать следующий подход:
missing_values = df.select([pl.col(column).is_null().sum().alias(f"missing_{column}") for column in df.columns])
print("\nКоличество пропусков в каждом столбце:")
print(missing_values)

# Для получения средних значений (для числовых столбцов)
print("\nСредние значения числовых столбцов:")
mean_values = df.select(pl.all().exclude(pl.Boolean).mean())
print(mean_values)


Типы данных столбцов:
OrderedDict([('PassengerId', Int64), ('Survived', Int64), ('Pclass', Int64), ('Name', Utf8), ('Sex', Utf8), ('Age', Float64), ('SibSp', Int64), ('Parch', Int64), ('Ticket', Utf8), ('Fare', Float64), ('Cabin', Utf8), ('Embarked', Utf8)])

Общая информация о датасете:
shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ describe   ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin ┆ Embarked │
│ ---        ┆ ---         ┆ ---      ┆ ---      ┆   ┆ ---       ┆ ---       ┆ ---   ┆ ---      │
│ str        ┆ f64         ┆ f64      ┆ f64      ┆   ┆ str       ┆ f64       ┆ str   ┆ str      │
╞════════════╪═════════════╪══════════╪══════════╪═══╪═══════════╪═══════════╪═══════╪══════════╡
│ count      ┆ 891.0       ┆ 891.0    ┆ 891.0    ┆ … ┆ 891       ┆ 891.0     ┆ 204   ┆ 889      │
│ null_count ┆ 0.0         ┆ 0.0      ┆ 0.0      ┆ … ┆ 0         ┆ 0.0       ┆ 687   ┆ 2        │
│ mean    

In [ ]:
passengers_per_class = df.groupby("Pclass").agg([
    pl.count("PassengerId").alias("Count")
]).sort("Pclass")

print(passengers_per_class)

shape: (3, 2)
┌────────┬───────┐
│ Pclass ┆ Count │
│ ---    ┆ ---   │
│ i64    ┆ u32   │
╞════════╪═══════╡
│ 1      ┆ 216   │
│ 2      ┆ 184   │
│ 3      ┆ 491   │
└────────┴───────┘


<ipython-input-16-95ca42d4fbfb>:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  passengers_per_class = df.groupby("Pclass").agg([


In [ ]:
print(df.columns)

['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [ ]:
# Подсчет количества выживших мужчин и женщин
survivors_by_gender = df.filter(pl.col("Survived") == 1).groupby("Sex").agg([
    pl.count("Survived").alias("SurvivedCount")
])

print(survivors_by_gender)

shape: (2, 2)
┌────────┬───────────────┐
│ Sex    ┆ SurvivedCount │
│ ---    ┆ ---           │
│ str    ┆ u32           │
╞════════╪═══════════════╡
│ male   ┆ 109           │
│ female ┆ 233           │
└────────┴───────────────┘


<ipython-input-18-c1e5c588c0d4>:2: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  survivors_by_gender = df.filter(pl.col("Survived") == 1).groupby("Sex").agg([


In [ ]:
# Фильтрация пассажиров, возраст которых больше 44 лет
passengers_over_44 = df.filter(pl.col("Age") > 44)

print(passengers_over_44)

shape: (115, 12)
┌─────────────┬──────────┬────────┬──────────────────┬───┬────────────┬─────────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name             ┆ … ┆ Ticket     ┆ Fare    ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---              ┆   ┆ ---        ┆ ---     ┆ ---   ┆ ---      │
│ i64         ┆ i64      ┆ i64    ┆ str              ┆   ┆ str        ┆ f64     ┆ str   ┆ str      │
╞═════════════╪══════════╪════════╪══════════════════╪═══╪════════════╪═════════╪═══════╪══════════╡
│ 7           ┆ 0        ┆ 1      ┆ McCarthy, Mr.    ┆ … ┆ 17463      ┆ 51.8625 ┆ E46   ┆ S        │
│             ┆          ┆        ┆ Timothy J        ┆   ┆            ┆         ┆       ┆          │
│ 12          ┆ 1        ┆ 1      ┆ Bonnell, Miss.   ┆ … ┆ 113783     ┆ 26.55   ┆ C103  ┆ S        │
│             ┆          ┆        ┆ Elizabeth        ┆   ┆            ┆         ┆       ┆          │
│ 16          ┆ 1        ┆ 2      ┆ Hewlett, Mrs.    ┆ … ┆ 248706     ┆ 16

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
!pip install bottleneck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 5.4 MB/s eta 0:00:00


In [ ]:
import bottleneck as bn

# Предполагаем, что столбец с возрастом называется 'Age'
ages = df['Age'].values  # Получаем numpy массив значений возраста

# Удаляем NaN значения для корректного расчета
ages_nonan = ages[~pd.isnull(ages)]

# Расчет среднего возраста и стандартного отклонения
mean_age = bn.nanmean(ages_nonan)  # bn.nanmean автоматически игнорирует NaN, но мы уже очистили данные от NaN
std_age = bn.nanstd(ages_nonan)

print(f"Средний возраст пассажиров: {mean_age}")
print(f"Стандартное отклонение возраста: {std_age}")

Средний возраст пассажиров: 29.69911764705882
Стандартное отклонение возраста: 14.516321150817317


In [ ]:
# Расчет нового значения Fare
df['Fare_new'] = df.apply(lambda row: row['Fare'] * 1.3 if row['Pclass'] in [1, 2] else row['Fare'] * 1.1, axis=1)

print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [ ]:
# Расчет нового значения Fare
df['Fare_new'] = np.where(df['Pclass'].isin([1, 2]), df['Fare'] * 1.3, df['Fare'] * 1.1)

print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [ ]:
df = pd.read_csv("housing.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


Перед изменением типов данных важно проверить наличие пропущенных значений (NaN), так как преобразование в целочисленные типы (int) невозможно, если столбец содержит NaN.

Большинство столбцов имеют тип данных float64. Если точность float64 не критична для анализа, можно рассмотреть уменьшение до float32, что сократит потребление памяти вдвое.

Данные столбцы могут быть преобразованы в float32, если потеря точности в последних знаках не существенна.

Для столбцов с типом данных int64, который занимает 8 байт на значение, можно рассмотреть преобразование в типы с меньшим размером (int32, int16 или даже int8), основываясь на максимальном и минимальном значениях данных в этих столбцах.

CHAS, RAD: Эти столбцы, содержат ограниченный набор уникальных значений. Например, CHAS может быть индикатором (0 или 1), и его можно преобразовать в int8. RAD также можно уменьшить до int8 или int16.

Рекомендации:
CRIM, ZN, INDUS, NOX, RM, AGE, DIS, TAX, PTRATIO, B, LSTAT, MEDV: преобразование в float32.
CHAS: если это поле принимает только два значения, например, 0 и 1, его можно преобразовать в int8.
RAD: нужно проверить диапазон значений, но, скорее всего, его можно безопасно преобразовать в int16 или даже int8, если диапазон позволяет.

In [ ]:
missing_values = df.isnull().sum()

print(missing_values)

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64


In [ ]:
has_missing_values = df.isnull().any().any()

print(has_missing_values)

False


In [ ]:
df = df.astype({
    'CRIM': 'float32',
    'ZN': 'float32',
    'INDUS': 'float32',
    'NOX': 'float32',
    'RM': 'float32',
    'AGE': 'float32',
    'DIS': 'float32',
    'TAX': 'float32',
    'PTRATIO': 'float32',
    'B': 'float32',
    'LSTAT': 'float32',
    'MEDV': 'float32',
    'CHAS': 'int8',
    'RAD': 'int16'
})

In [ ]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float32
 1   ZN       506 non-null    float32
 2   INDUS    506 non-null    float32
 3   CHAS     506 non-null    int8   
 4   NOX      506 non-null    float32
 5   RM       506 non-null    float32
 6   AGE      506 non-null    float32
 7   DIS      506 non-null    float32
 8   RAD      506 non-null    int16  
 9   TAX      506 non-null    float32
 10  PTRATIO  506 non-null    float32
 11  B        506 non-null    float32
 12  LSTAT    506 non-null    float32
 13  MEDV     506 non-null    float32
dtypes: float32(12), int16(1), int8(1)
memory usage: 25.3 KB
